In [1]:
model_path = "craft_mlt_25k.pth"

In [2]:
import torch
import onnx
from craft import CRAFT # Class from Craft Repository (https://github.com/clovaai/CRAFT-pytorch) 
from collections import OrderedDict

In [3]:
def copyStateDict(state_dict):
    if list(state_dict.keys())[0].startswith("module"):
        start_idx = 1
    else:
        start_idx = 0
    new_state_dict = OrderedDict()
    for k, v in state_dict.items():
        name = ".".join(k.split(".")[start_idx:])
        new_state_dict[name] = v
    return new_state_dict

In [4]:
# Load the model and its weights
model = CRAFT()
model.load_state_dict(copyStateDict(torch.load(model_path, map_location='cpu')))

# Set the model to evaluation mode
model.eval()

CRAFT(
  (basenet): vgg16_bn(
    (slice1): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU(inplace=True)
      (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (9): ReLU(inplace=True)
      (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (slice2): Sequential(
      (12): ReLU(inplace=True)
      (13): MaxPool2d(kerne

In [5]:
dummy_input = torch.randn(1, 3, 768, 768, device="cpu")

In [6]:
input_names = [ "actual_input_1" ] + [ "learned_%d" % i for i in range(16) ]
output_names = [ "output1" ]

In [7]:
# torch.onnx.export(model, dummy_input, "craft_mlt_25k-verbose-true.onnx", verbose=True, input_names=input_names, output_names=output_names)
# torch.onnx.export(model, dummy_input, "craft_mlt_25k-2.onnx", input_names=input_names, output_names=output_names)
torch.onnx.export(model, dummy_input, "craft_mlt_25k-opset-11.onnx", verbose=True, input_names=input_names, output_names=output_names, opset_version=11)

graph(%actual_input_1 : Float(1, 3, 768, 768, strides=[1769472, 589824, 768, 1], requires_grad=0, device=cpu),
      %basenet.slice5.1.weight : Float(1024, 512, 3, 3, strides=[4608, 9, 3, 1], requires_grad=1, device=cpu),
      %basenet.slice5.1.bias : Float(1024, strides=[1], requires_grad=1, device=cpu),
      %basenet.slice5.2.weight : Float(1024, 1024, 1, 1, strides=[1024, 1, 1, 1], requires_grad=1, device=cpu),
      %basenet.slice5.2.bias : Float(1024, strides=[1], requires_grad=1, device=cpu),
      %conv_cls.0.weight : Float(32, 32, 3, 3, strides=[288, 9, 3, 1], requires_grad=1, device=cpu),
      %conv_cls.0.bias : Float(32, strides=[1], requires_grad=1, device=cpu),
      %conv_cls.2.weight : Float(32, 32, 3, 3, strides=[288, 9, 3, 1], requires_grad=1, device=cpu),
      %conv_cls.2.bias : Float(32, strides=[1], requires_grad=1, device=cpu),
      %conv_cls.4.weight : Float(16, 32, 3, 3, strides=[288, 9, 3, 1], requires_grad=1, device=cpu),
      %conv_cls.4.bias : Float(16, 